In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

#DATA

In [ ]:
#SPACY NER -> the accuracy is unacceptable 
# FOR FRENCH
#!python -m spacy download fr_core_news_lg
# FOR SPANISH
#!python -m spacy download es_core_news_lg
# FOR GERMAN
#!python -m spacy download de_core_news_lg
# FOR RUSSIAN
#!python -m spacy download ru_core_news_lg

In [2]:
import os
import re
import csv
#import spacy
import argparse
import sys
from google.colab import files

In [3]:
!pip install stanza
import stanza

#stanza.download('fr') # FRENCH
#stanza.download('es')  # SPANISH
stanza.download('de') # GERMAN
#stanza.download('ru') # RUSSIAN

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=e739f047a213a9eb01a6600f36af6f9f95625bb02a6d1fb52ce8ff168c98f033
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


INFO:stanza:Downloading default packages for language: de (German) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [4]:
# Create the Input folder
if not os.path.exists('/content/Input'):
    os.makedirs('/content/Input')

# Create the Output folder
if not os.path.exists('/content/Output'):
    os.makedirs('/content/Output')

# Create the Output folder
if not os.path.exists('/content/Output/Output2'):
    os.makedirs('/content/Output/Output2')

# Create the Output folder
if not os.path.exists('/content/Output/Output3'):
    os.makedirs('/content/Output/Output3')

# Create the Output folder
if not os.path.exists('/content/Output/Output4'):
    os.makedirs('/content/Output/Output4')

# Upload the files
uploaded_files = files.upload()

# Set the input directory
input_dir = '/content/Input'

# Iterate over the uploaded files
for file_name, data in uploaded_files.items():
    # Construct the input file path
    input_path = os.path.join(input_dir, file_name)
    
    # Write the uploaded file to the input directory
    with open(input_path, 'wb') as f:
        f.write(data)

Saving de_gsd-ud_CLEAN.conllu to de_gsd-ud_CLEAN.conllu
Saving de_pud-ud_CLEAN.conllu to de_pud-ud_CLEAN.conllu


#Truecaser

In [ ]:
def extract_words(input_file, output_file):
    with open(input_file, "r") as file, open(output_file, "w") as outfile:
        for line in file:
            if line.startswith("#") or line.strip() == "":
                continue
            columns = line.strip().split("\t")
            if len(columns) < 4:
                continue
            word = columns[1].lower()
            if columns[3] == "PROPER_NOUN":
                outfile.write(word + "\tINIT_UPPER\n")
            else:
                outfile.write(word + "\tSKIP\n")
    print(f"File {output_file} has been created successfully.")

directory = os.path.abspath("/content/Input")

for file in os.listdir(directory):
    if file.endswith(".conllu"):
        input_file = os.path.join(directory, file)
        output_file = os.path.join(directory, file.replace(".conllu", "_truecaser.tsv"))
        extract_words(input_file, output_file)

print("All files has been processed successfully.")

File /content/Input/es_gsd-ud_CLEAN_truecaser.tsv has been created successfully.
File /content/Input/es_ancora-ud_CLEAN_truecaser.tsv has been created successfully.
File /content/Input/es_pud-ud_CLEAN_truecaser.tsv has been created successfully.
All files has been processed successfully.


In [ ]:
from google.colab import files
def download_xml_files():
    # Set the input and output directories
    input_dir = '/content/Input'

    # Iterate over the .conllu files in the input directory
    for filename in os.listdir(input_dir):
        if not filename.endswith('.tsv'):
            continue

        # Construct the full file path
        file_path = os.path.join(input_dir, filename)

        # Download the file
        files.download(file_path)

download_xml_files()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#NER

###ORIGINAL WAY

In [ ]:
# load the spacy french model
#nlp = spacy.load('fr_core_news_lg') #FRENCH
#nlp = spacy.load('es_core_news_lg') #SPANISH
#nlp = spacy.load('de_core_news_lg') #GERMAN
#nlp = spacy.load('ru_core_news_lg) #RUSSIAN

def extract_words(input_file, output_file):
    with open(input_file, "r") as file, open(output_file, "w") as outfile:
        for line in file:
            if line.startswith("#") or line.strip() == "":
                continue
            columns = line.strip().split("\t")
            if len(columns) < 4:
                continue
            word = columns[1]
            if columns[3] != "PROPER_NOUN":
                outfile.write(word + "\tO\t" + columns[3] + "\n")
            else:
                doc = nlp(word)
                for ent in doc.ents:
                    if ent.label_ in ["PER", "ORG", "LOC"]:
                        outfile.write(word + "\t" + ent.label_  + "\t" +  columns[3] + "\n")

directory = os.path.abspath("/content/Input")
odirectory = os.path.abspath("/content/Output")

for file in os.listdir(directory):
    if file.endswith(".conllu"):
        input_file = os.path.join(directory, file)
        output_file = os.path.join(odirectory, file.replace(".conllu", "_NER.tsv"))
        extract_words(input_file, output_file)

print("All files has been processed successfully.")

All files has been processed successfully.


In [ ]:
#nlp = spacy.load('fr_core_news_lg') #FRENCH
def extract_words(input_file, output_file):
    with open(input_file, "r") as file, open(output_file, "w") as outfile:
        for line in file:
            if line.startswith("#") or line.strip() == "":
                continue
            columns = line.strip().split("\t")
            if len(columns) < 4:
                continue
            word = columns[1]
            is_written = False
            if columns[3] != "PROPER_NOUN":
                outfile.write(word + "\tO\t" + columns[3] + "\n")
                is_written = True
            else:
                doc = nlp(word)
                for ent in doc.ents:
                    if ent.label_ in ["PER", "ORG", "LOC"]:
                        outfile.write(word + "\t" + ent.label_  + "\t" +  columns[3] + "\n")
                        is_written = True
            if not is_written:
                outfile.write(word + "\t???\t" + columns[3] + "\n")

directory = os.path.abspath("/content/Input")
odirectory = os.path.abspath("/content/Output")

for file in os.listdir(directory):
    if file.endswith(".conllu"):
        input_file = os.path.join(directory, file)
        output_file = os.path.join(odirectory, file.replace(".conllu", "_NER.tsv"))
        extract_words(input_file, output_file)

print("All files has been processed successfully.")

AttributeError: ignored

#### ALTERNATIVE WITH TEST

In [ ]:
#     FRENCH:            #      SPANISH           #     RUSSIAN         #     GERMAN
# FQB - 335/2071         # Ancora - 5534/19999+   # Taiga - 758/ 4000   # HDT - 
# Sequoia - 426/3316     # PUD -  38/1235         # PUD - 113/1160      # PUD - 53/1220
# Rhapsodie - 172/1015   # GSD -  7112/19999+     # GSD - 415/6657      # GSD - 9269//19999+
# PUD - 121/1272
# Parisstories - 106/434

# Initialize the NLP pipeline using the French model
#nlp = stanza.Pipeline('fr')
#nlp = stanza.Pipeline('es') 
#nlp = stanza.Pipeline('de')
#nlp = stanza.Pipeline('ru')

def extract_words(input_file, output_file):
    with open(input_file, "r") as file, open(output_file, "w") as outfile:
        for line in file:
            if line.startswith("#") or line.strip() == "":
                continue
            columns = line.strip().split("\t")
            if len(columns) < 4:
                continue
            word = columns[1]
            is_written = False
            if columns[3] != "PROPER_NOUN":
                outfile.write(word + "\tO\t" + columns[3] + "\n")
                is_written = True
            else:
                doc = nlp(word)
                for sent in doc.sentences:
                    for ent in sent.ents:
                        if ent.type in ["PER", "ORG", "LOC"]:
                            outfile.write(word + "\t" + ent.type  + "\t" +  columns[3] + "\n")
                            is_written = True
            if not is_written:
                outfile.write(word + "\t???\t" + columns[3] + "\n")

directory = os.path.abspath("/content/Input")
odirectory = os.path.abspath("/content/Output")

for file in os.listdir(directory):
    if file.endswith(".conllu"):
        input_file = os.path.join(directory, file)
        output_file = os.path.join(odirectory, file.replace(".conllu", "_NER.tsv"))
        extract_words(input_file, output_file)

print("All files has been processed successfully.")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: es (Spanish):
| Processor    | Package  |
---------------------------
| tokenize     | ancora   |
| mwt          | ancora   |
| pos          | ancora   |
| lemma        | ancora   |
| depparse     | ancora   |
| sentiment    | tass2020 |
| constituency | combined |
| ner          | conll02  |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: constituency
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


### WITH ID

#### stanza

In [6]:
# Initialize the NLP pipeline using the French model
#nlp = stanza.Pipeline('fr', processors='tokenize,ner')
#nlp = stanza.Pipeline('es', processors='tokenize,ner') 
nlp = stanza.Pipeline('de', processors='tokenize,ner')
#nlp = stanza.Pipeline('ru', processors='tokenize,ner')

def extract_words(input_file, output_file):
    with open(input_file, "r") as file, open(output_file, "w") as outfile:
        for line in file:
            if line.startswith("# id = "):
                outfile.write(line)
            if line.startswith("#") or line.strip() == "":
                continue
            columns = line.strip().split("\t")
            if len(columns) < 4:
                continue
            word = columns[1]
            is_written = False
            if columns[3] != "PROPER_NOUN":
                outfile.write(word + "\tO\t" + columns[3] + "\n")
                is_written = True
            else:
                doc = nlp(word)
                for sent in doc.sentences:
                    for ent in sent.ents:
                        if ent.type in ["PER", "ORG", "LOC"]:
                            outfile.write(word + "\t" + ent.type  + "\t" +  columns[3] + "\n")
                            is_written = True
            if not is_written:
                outfile.write(word + "\t???\t" + columns[3] + "\n")

directory = os.path.abspath("/content/Input")
odirectory = os.path.abspath("/content/Output")

for file in os.listdir(directory):
    if file.endswith(".conllu"):
        input_file = os.path.join(directory, file)
        output_file = os.path.join(odirectory, file.replace(".conllu", "_NER.tsv"))
        extract_words(input_file, output_file)

print("All files has been processed successfully.")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: de (German):
| Processor | Package      |
----------------------------
| tokenize  | gsd          |
| mwt       | gsd          |
| ner       | germeval2014 |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


All files has been processed successfully.


#### Let's eliminate lines

In [7]:
def eliminate_lines(input_file, output_file):
    with open(input_file, "r") as file:
        lines = file.readlines()
    with open(output_file, "w") as outfile:
        id = None
        eliminated = []
        for line in lines:
            if line.startswith("# id = "):
                id = int(line.strip().split(" ")[-1])
                outfile.write(line)
                continue
            if line.strip().split("\t")[1] != "???":
                outfile.write(line)
            else:
                if id:
                    eliminated.append(id)
                else:
                    outfile.write(line)
    print(f"{len(eliminated)} # id = lines were eliminated from file '{os.path.basename(input_file)}': {eliminated}")

def delete_hash_id_lines(input_file, output_file):
  with open(input_file, "r") as file:
    lines = file.readlines()
    with open(output_file, "w") as outfile:
      for line in lines:
        if not line.startswith("# id = "):
          outfile.write(line)
        else:
          outfile.write("\n")
    print(f"All lines starting with '# id = ' have been eliminated from file '{os.path.basename(input_file)}'.")

input_directory = os.path.abspath("/content/Output")
output_directory = os.path.abspath("/content/Output/Output2")

for file in os.listdir(input_directory):
    if file.endswith("_NER.tsv"):
        input_file = os.path.join(input_directory, file)
        output_file = os.path.join(output_directory, file)
        eliminate_lines(input_file, output_file)
        delete_hash_id_lines(output_file, output_file)

print("All files have been processed successfully.")

47584 # id = lines were eliminated from file 'de_hdt-ud_CLEAN_NER.tsv': [4, 10, 13, 28, 29, 30, 34, 35, 36, 54, 56, 56, 60, 63, 105, 105, 106, 113, 114, 120, 122, 125, 130, 157, 157, 158, 160, 160, 161, 161, 163, 175, 175, 187, 187, 194, 194, 201, 203, 205, 207, 208, 209, 212, 215, 216, 218, 222, 223, 223, 224, 226, 226, 227, 228, 230, 240, 241, 263, 264, 270, 271, 275, 276, 278, 279, 280, 282, 284, 284, 286, 296, 307, 308, 313, 313, 313, 316, 320, 326, 343, 344, 345, 345, 349, 351, 353, 361, 362, 363, 364, 370, 372, 372, 376, 376, 382, 402, 402, 406, 406, 417, 430, 433, 435, 446, 446, 447, 452, 453, 454, 457, 459, 465, 469, 469, 475, 483, 491, 497, 498, 502, 503, 503, 504, 505, 506, 511, 515, 528, 528, 528, 530, 533, 534, 537, 537, 539, 542, 543, 554, 560, 563, 581, 597, 605, 605, 606, 606, 606, 612, 613, 614, 615, 616, 617, 617, 618, 618, 619, 621, 621, 622, 623, 625, 625, 627, 628, 640, 641, 643, 644, 654, 657, 666, 666, 670, 672, 679, 699, 699, 716, 716, 762, 764, 765, 765, 766, 76

#### Let's apply :START :END and :INTERMEDIATE

In [8]:
input_dir = "/content/Output/Output2"
output_dir = "/content/Output/Output3"

# Loop through files in input_dir
for filename in os.listdir(input_dir):
    file_path = os.path.join(input_dir, filename)
    
    # Skip if file is a directory
    if os.path.isdir(file_path):
        continue

    with open(file_path, 'r') as f:
        lines = f.readlines()

    # change the tags
    for i in range(len(lines)):
        lines[i] = re.sub(r'\tPER\t', '\tPERSON\t', lines[i])
        lines[i] = re.sub(r'\tLOC\t', '\tLOCATION\t', lines[i])
        lines[i] = re.sub(r'\tORG\t', '\tORGANIZATION\t', lines[i])

    with open(os.path.join(output_dir, filename), 'w') as f:
        f.writelines(lines)


####PERSON

In [9]:
input_dir = "/content/Output/Output3"
output_dir = "/content/Output/Output4"

for file in os.listdir(input_dir):
    if file.endswith(".tsv"):
        filename = file.replace(".tsv", ".tsv")
        with open(os.path.join(input_dir, file), "r", encoding="utf-8") as f:
            lines = f.readlines()

        for i in range(len(lines)):
          if lines[i] == "\n" or lines[i-1] == "\n" or (i+1 < len(lines) and lines[i+1] == "\n"):
            continue
          elif lines[i].split('\t')[1] == 'PERSON' and (i == 0 or lines[i-1].split('\t')[1] != 'PERSON') and (i + 1) < len(lines) and lines[i + 1].split('\t')[1] == 'PERSON':
            lines[i] = re.sub(r'PERSON', 'PERSON:START', lines[i])
          elif (lines[i].split('\t')[1] == 'PERSON' or lines[i].split('\t')[1] == 'PERSON:START') and (i - 1) >= 0 and (lines[i-1].split('\t')[1] == 'PERSON' or lines[i-1].split('\t')[1] == 'PERSON:START') and (i + 1) < len(lines) and lines[i + 1].split('\t')[1] != 'PERSON':
            lines[i] = re.sub(r'PERSON', 'PERSON:END', lines[i])
        
        empty_start = 1
        while empty_start > 0:
          empty_start = 0
          for i in range(len(lines)):
            if lines[i] == "\n" or (i+1 < len(lines) and lines[i+1] == "\n"):
              continue
            elif lines[i].split('\t')[1] == 'PERSON' and (i == 0 or lines[i-1] == "\n") and (i + 1) < len(lines) and lines[i + 1].split('\t')[1] == 'PERSON':
              lines[i] = re.sub(r'PERSON', 'PERSON:START', lines[i])
              empty_start += 1

        empty_end = 1
        while empty_end > 0:
          empty_end = 0
          for i in range(len(lines)):
            if lines[i] == "\n" or lines[i-1] == "\n":
              continue
            elif (lines[i].split('\t')[1] == 'PERSON' or lines[i].split('\t')[1] == 'PERSON:START') and (i - 1) >= 0 and (lines[i-1].split('\t')[1] == 'PERSON' or lines[i-1].split('\t')[1] == 'PERSON:START') and (i+1 < len(lines) and lines[i+1] == "\n"):
              lines[i] = re.sub(r'PERSON', 'PERSON:END', lines[i])
              empty_end += 1        
        

        changes = 1
        while changes > 0:
          changes = 0
          for i in range(len(lines)):
            if lines[i] == "\n" or lines[i-1] == "\n" or (i+1 < len(lines) and lines[i+1] == "\n"):
              continue
            elif (lines[i].split('\t')[1] == 'PERSON:START') and (i - 1) >= 0 and (lines[i-1].split('\t')[1] == 'PERSON:START') and (i + 1) < len(lines) and (lines[i + 1].split('\t')[1] == 'PERSON:END' or lines[i + 1].split('\t')[1] == 'PERSON:INTERMEDIATE'):
              lines[i] = re.sub(r'PERSON:START', 'PERSON:INTERMEDIATE', lines[i])
              changes += 1

        with open(os.path.join(output_dir, filename), 'w') as f:
          f.writelines(lines)


####LOCATION

In [10]:
input_dir = "/content/Output/Output4"
output_dir = "/content/Output/Output4"

for file in os.listdir(input_dir):
    if file.endswith(".tsv"):
        filename = file.replace(".tsv", ".tsv")
        with open(os.path.join(input_dir, file), "r", encoding="utf-8") as f:
            lines = f.readlines()

        for i in range(len(lines)):
          if lines[i] == "\n" or lines[i-1] == "\n" or (i+1 < len(lines) and lines[i+1] == "\n"):
            continue
          elif lines[i].split('\t')[1] == 'LOCATION' and (i == 0 or lines[i-1].split('\t')[1] != 'LOCATION') and (i + 1) < len(lines) and lines[i + 1].split('\t')[1] == 'LOCATION':
            lines[i] = re.sub(r'LOCATION', 'LOCATION:START', lines[i])
          elif (lines[i].split('\t')[1] == 'LOCATION' or lines[i].split('\t')[1] == 'LOCATION:START') and (i - 1) >= 0 and (lines[i-1].split('\t')[1] == 'LOCATION' or lines[i-1].split('\t')[1] == 'LOCATION:START') and (i + 1) < len(lines) and lines[i + 1].split('\t')[1] != 'LOCATION':
            lines[i] = re.sub(r'LOCATION', 'LOCATION:END', lines[i])
        
        empty_start = 1
        while empty_start > 0:
          empty_start = 0
          for i in range(len(lines)):
            if lines[i] == "\n" or (i+1 < len(lines) and lines[i+1] == "\n"):
              continue
            elif lines[i].split('\t')[1] == 'LOCATION' and (i == 0 or lines[i-1] == "\n") and (i + 1) < len(lines) and lines[i + 1].split('\t')[1] == 'LOCATION':
              lines[i] = re.sub(r'LOCATION', 'LOCATION:START', lines[i])
              empty_start += 1

        empty_end = 1
        while empty_end > 0:
          empty_end = 0
          for i in range(len(lines)):
            if lines[i] == "\n" or lines[i-1] == "\n":
              continue
            elif (lines[i].split('\t')[1] == 'LOCATION' or lines[i].split('\t')[1] == 'LOCATION:START') and (i - 1) >= 0 and (lines[i-1].split('\t')[1] == 'LOCATION' or lines[i-1].split('\t')[1] == 'LOCATION:START') and (i+1 < len(lines) and lines[i+1] == "\n"):
              lines[i] = re.sub(r'LOCATION', 'LOCATION:END', lines[i])
              empty_end += 1        
        

        changes = 1
        while changes > 0:
          changes = 0
          for i in range(len(lines)):
            if lines[i] == "\n" or lines[i-1] == "\n" or (i+1 < len(lines) and lines[i+1] == "\n"):
              continue
            elif (lines[i].split('\t')[1] == 'LOCATION:START') and (i - 1) >= 0 and (lines[i-1].split('\t')[1] == 'LOCATION:START') and (i + 1) < len(lines) and (lines[i + 1].split('\t')[1] == 'LOCATION:END' or lines[i + 1].split('\t')[1] == 'LOCATION:INTERMEDIATE'):
              lines[i] = re.sub(r'LOCATION:START', 'LOCATION:INTERMEDIATE', lines[i])
              changes += 1

        with open(os.path.join(output_dir, filename), 'w') as f:
          f.writelines(lines)

####ORGANIZATION

In [11]:
input_dir = "/content/Output/Output4"
output_dir = "/content/Output/Output4"

for file in os.listdir(input_dir):
    if file.endswith(".tsv"):
        filename = file.replace(".tsv", ".tsv")
        with open(os.path.join(input_dir, file), "r", encoding="utf-8") as f:
            lines = f.readlines()

        for i in range(len(lines)):
          if lines[i] == "\n" or lines[i-1] == "\n" or (i+1 < len(lines) and lines[i+1] == "\n"):
            continue
          elif lines[i].split('\t')[1] == 'ORGANIZATION' and (i == 0 or lines[i-1].split('\t')[1] != 'ORGANIZATION') and (i + 1) < len(lines) and lines[i + 1].split('\t')[1] == 'ORGANIZATION':
            lines[i] = re.sub(r'ORGANIZATION', 'ORGANIZATION:START', lines[i])
          elif (lines[i].split('\t')[1] == 'ORGANIZATION' or lines[i].split('\t')[1] == 'ORGANIZATION:START') and (i - 1) >= 0 and (lines[i-1].split('\t')[1] == 'ORGANIZATION' or lines[i-1].split('\t')[1] == 'ORGANIZATION:START') and (i + 1) < len(lines) and lines[i + 1].split('\t')[1] != 'ORGANIZATION':
            lines[i] = re.sub(r'ORGANIZATION', 'ORGANIZATION:END', lines[i])
        
        empty_start = 1
        while empty_start > 0:
          empty_start = 0
          for i in range(len(lines)):
            if lines[i] == "\n" or (i+1 < len(lines) and lines[i+1] == "\n"):
              continue
            elif lines[i].split('\t')[1] == 'ORGANIZATION' and (i == 0 or lines[i-1] == "\n") and (i + 1) < len(lines) and lines[i + 1].split('\t')[1] == 'ORGANIZATION':
              lines[i] = re.sub(r'ORGANIZATION', 'ORGANIZATION:START', lines[i])
              empty_start += 1

        empty_end = 1
        while empty_end > 0:
          empty_end = 0
          for i in range(len(lines)):
            if lines[i] == "\n" or lines[i-1] == "\n":
              continue
            elif (lines[i].split('\t')[1] == 'ORGANIZATION' or lines[i].split('\t')[1] == 'ORGANIZATION:START') and (i - 1) >= 0 and (lines[i-1].split('\t')[1] == 'ORGANIZATION' or lines[i-1].split('\t')[1] == 'ORGANIZATION:START') and (i+1 < len(lines) and lines[i+1] == "\n"):
              lines[i] = re.sub(r'ORGANIZATION', 'ORGANIZATION:END', lines[i])
              empty_end += 1        
        

        changes = 1
        while changes > 0:
          changes = 0
          for i in range(len(lines)):
            if lines[i] == "\n" or lines[i-1] == "\n" or (i+1 < len(lines) and lines[i+1] == "\n"):
              continue
            elif (lines[i].split('\t')[1] == 'ORGANIZATION:START') and (i - 1) >= 0 and (lines[i-1].split('\t')[1] == 'ORGANIZATION:START') and (i + 1) < len(lines) and (lines[i + 1].split('\t')[1] == 'ORGANIZATION:END' or lines[i + 1].split('\t')[1] == 'ORGANIZATION:INTERMEDIATE'):
              lines[i] = re.sub(r'ORGANIZATION:START', 'ORGANIZATION:INTERMEDIATE', lines[i])
              changes += 1

        with open(os.path.join(output_dir, filename), 'w') as f:
          f.writelines(lines)

#### DOWNLOAD

In [12]:
from google.colab import files
def download_xml_files():
    # Set the input and output directories
    input_dir = '/content/Output/Output4'

    # Iterate over the .conllu files in the input directory
    for filename in os.listdir(input_dir):
        if not filename.endswith('.tsv'):
            continue

        # Construct the full file path
        file_path = os.path.join(input_dir, filename)

        # Download the file
        files.download(file_path)

download_xml_files()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Archive

In [51]:
input_dir = "/content/Output/Output3"
output_dir = "/content/Output/Output4"

for file in os.listdir(input_dir):
    if file.endswith(".tsv"):
        filename = file.replace(".tsv", ".tsv")
        with open(os.path.join(input_dir, file), "r", encoding="utf-8") as f:
            lines = f.readlines()

        for i in range(len(lines)):
          if lines[i].split('\t')[1] == 'PERSON' and (i == 0 or lines[i-1].split('\t')[1] != 'PERSON') and (i + 1) < len(lines) and lines[i + 1].split('\t')[1] == 'PERSON':
            lines[i] = re.sub(r'PERSON', 'PERSON:START', lines[i])
          elif (lines[i].split('\t')[1] == 'PERSON' or lines[i].split('\t')[1] == 'PERSON:START') and (i - 1) >= 0 and (lines[i-1].split('\t')[1] == 'PERSON' or lines[i-1].split('\t')[1] == 'PERSON:START') and (i + 1) < len(lines) and lines[i + 1].split('\t')[1] != 'PERSON':
            lines[i] = re.sub(r'PERSON', 'PERSON:END', lines[i])
        changes = 1
        while changes > 0:
          changes = 0
          for i in range(len(lines)):
            if (lines[i].split('\t')[1] == 'PERSON:START') and (i - 1) >= 0 and (lines[i-1].split('\t')[1] == 'PERSON:START') and (i + 1) < len(lines) and (lines[i + 1].split('\t')[1] == 'PERSON:END' or lines[i + 1].split('\t')[1] == 'PERSON:INTERMEDIATE'):
              lines[i] = re.sub(r'PERSON:START', 'PERSON:INTERMEDIATE', lines[i])
              changes += 1

        with open(os.path.join(output_dir, filename), 'w') as f:
          f.writelines(lines)


IndexError: ignored

In [ ]:
def add_ner_labels(input_file):
    with open(input_file, "r") as file, open(input_file+"_labeled.tsv", "w") as outfile:
        previous_tag = ""
        count = 0
        for line in file:
            word, tag, proper_noun = line.strip().split("\t")
            if tag == "PER":
                if previous_tag != tag:
                    count = 1
                    outfile.write(word + "\tPERSON:START\t" + proper_noun + "\n")
                elif previous_tag == tag:
                    if count == 1:
                        outfile.write(word + "\tPERSON:END\t" + proper_noun + "\n")
                    else:
                        outfile.write(word + "\tPERSON:INTERMEDIATE\t" + proper_noun + "\n")
                    count += 1
            elif tag == "ORG":
                if previous_tag != tag:
                    count = 1
                    outfile.write(word + "\tORGANIZATION:START\t" + proper_noun + "\n")
                elif previous_tag == tag:
                    if count == 1:
                        outfile.write(word + "\tORGANIZATION:END\t" + proper_noun + "\n")
                    else:
                        outfile.write(word + "\tORGANIZATION:INTERMEDIATE\t" + proper_noun + "\n")
                    count += 1
            elif tag == "LOC":
                if previous_tag != tag:
                    count = 1
                    outfile.write(word + "\tLOCATION:START\t" + proper_noun + "\n")
                elif previous_tag == tag:
                    if count == 1:
                        outfile.write(word + "\tLOCATION:END\t" + proper_noun + "\n")
                    else:
                        outfile.write(word + "\tLOCATION:INTERMEDIATE\t" + proper_noun + "\n")
                    count += 1
            else:
                outfile.write(line)
            previous_tag = tag
    print(f"File {input_file+'_labeled.tsv'} has been created successfully.")

directory = "/content/Output"
for filename in os.listdir(directory):
    if filename.endswith(".tsv"):
        input_file = os.path.join(directory, filename)
        add_ner_labels(input_file)

File /content/Output/fr_parisstories-ud_NER.tsv_labeled.tsv has been created successfully.
File /content/Output/fr_rhapsodie-ud_NER.tsv_labeled.tsv has been created successfully.
File /content/Output/fr_fqb-ud_NER.tsv_labeled.tsv has been created successfully.
File /content/Output/fr_sequoia-ud_NER.tsv_labeled.tsv has been created successfully.
File /content/Output/fr_pud-ud_NER.tsv_labeled.tsv has been created successfully.


In [ ]:
def adjust_tags(file_name):
    with open(file_name, 'r') as f_in:
        lines = f_in.readlines()
    with open(file_name, 'w') as f_out:
        for i in range(len(lines) - 1):
            word, tag, pos = lines[i].strip().split('\t')
            if tag.endswith(':START') and lines[i+1].strip().split('\t')[1] == 'O':
                tag = tag[:-6]
            f_out.write(f"{word}\t{tag}\t{pos}\n")

input_dir = '/content/Output/Output21/'
output_dir = '/content/Output/Output22/'

for file_name in os.listdir(input_dir):
    if file_name.endswith('.tsv'):
        adjust_tags(input_dir + file_name)
        os.rename(input_dir + file_name, output_dir + file_name)


##Test for :END :END [not necessary anymore]

In [ ]:
import sys
import os

def check_file(filename):
    double_count = 0
    triple_count = 0
    quadruple_count = 0
    line_numbers = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines) - 3):
            line1 = lines[i].strip().split('\t')[1]
            line2 = lines[i + 1].strip().split('\t')[1]
            line3 = lines[i + 2].strip().split('\t')[1]
            line4 = lines[i + 3].strip().split('\t')[1]
            if (line1 == "PERSON:END" and line2 == "PERSON:END" and line3 == "PERSON:END" and line4 == "PERSON:END") or \
               (line1 == "LOCATION:END" and line2 == "LOCATION:END" and line3 == "LOCATION:END" and line4 == "LOCATION:END") or \
               (line1 == "ORGANIZATION:END" and line2 == "ORGANIZATION:END" and line3 == "ORGANIZATION:END" and line4 == "ORGANIZATION:END"):
                quadruple_count += 1
                line_numbers.append(i + 1)
            elif (line1 == "PERSON:END" and line2 == "PERSON:END" and line3 == "PERSON:END") or \
                 (line1 == "LOCATION:END" and line2 == "LOCATION:END" and line3 == "LOCATION:END") or \
                 (line1 == "ORGANIZATION:END" and line2 == "ORGANIZATION:END" and line3 == "ORGANIZATION:END"):
                triple_count += 1
                line_numbers.append(i + 1)
            elif (line1 == "PERSON:END" and line2 == "PERSON:END") or \
                 (line1 == "LOCATION:END" and line2 == "LOCATION:END") or \
                 (line1 == "ORGANIZATION:END" and line2 == "ORGANIZATION:END"):
                double_count += 1
                line_numbers.append(i + 1)
    return double_count, triple_count, quadruple_count, line_numbers, filename

directory = "/content/Input"
for filename in os.listdir(directory):
    if filename.endswith(".tsv"):
        full_path = os.path.join(directory, filename)
        double_count, triple_count, quadruple_count, line_numbers, name = check_file(full_path)
        print(f"{name} - Quadruple cases: {quadruple_count} - Triple cases: {triple_count} - Double cases: {double_count}")
        if quadruple_count > 0:
            print(f"Line numbers for quadruple cases in {name}: {line_numbers}")

/content/Output/Output3/fr_sequoia-ud_NER.tsv -Double cases: 1
Line numbers for quadruple cases in /content/Output/Output3/fr_sequoia-ud_NER.tsv: [2027]
/content/Output/Output3/fr_pud-ud_NER.tsv -Double cases: 1
Line numbers for quadruple cases in /content/Output/Output3/fr_pud-ud_NER.tsv: [21329]
/content/Output/Output3/fr_parisstories-ud_NER.tsv -Double cases: 0
/content/Output/Output3/fr_fqb-ud_NER.tsv -Double cases: 2
Line numbers for quadruple cases in /content/Output/Output3/fr_fqb-ud_NER.tsv: [402, 8287]
/content/Output/Output3/fr_rhapsodie-ud_NER.tsv -Double cases: 0


In [24]:
import os

path = "/content/Output/Output4"
end_tags = ["PERSON:START"]

for filename in os.listdir(path):
  if filename.endswith(".tsv"):
    file_path = os.path.join(path, filename)
    with open(file_path, "r") as f:
      lines = f.readlines()
      triple_count = 0
      double_count = 0
      quadruple_count = 0
      for i, line in enumerate(lines):
        line = line.strip().split("\t")
        if line[1] in end_tags:
          if i + 3 < len(lines) and \
             all([l.strip().split("\t")[1] == line[1] for l in lines[i:i+4]]):
            quadruple_count += 1
            print(f"Quadruple case found in line {i + 1} in file {filename}")
          elif i + 1 < len(lines) and \
               lines[i + 1].strip().split("\t")[1] == line[1]:
            double_count += 1
      print(f"{filename} - triple cases: {triple_count} - double cases: {double_count} - quadruple cases: {quadruple_count}")


fr_fqb-ud_NER.tsv - triple cases: 0 - double cases: 0 - quadruple cases: 0
fr_rhapsodie-ud_NER.tsv - triple cases: 0 - double cases: 0 - quadruple cases: 0
fr_pud-ud_NER.tsv - triple cases: 0 - double cases: 0 - quadruple cases: 0
fr_sequoia-ud_NER.tsv - triple cases: 0 - double cases: 0 - quadruple cases: 0
fr_parisstories-ud_NER.tsv - triple cases: 0 - double cases: 0 - quadruple cases: 0
